# Test Functions

## Description
This notebook is done so that we can have a new way of testing our models, based on the validation set from Wider Face. This starts by restructuring the text file so that the calculations to check if it's correct or not is simpler, and we get the number of True Poistives, False Positives and False Negatives in a very easy way.

In [26]:
import cv2

In [34]:
def rewrite_test_file(original_dir, images_dir, new_dir, new_images_dir, limit_faces):
    counter = 0
    with open(original_dir, "r") as reader, open(new_dir, "w") as writer:
        while True:
            # First line is the directory to the image, we need to remove the folder it's in, as all images will be together
            file_dir = reader.readline()
            img_name = file_dir.split("/")[1]
            writer.write(img_name)
            
            # Get the image in the directory
            img = cv2.imread(images_dir + file_dir[:-1])
            
            #Write it on the new images' directory, under the saved name
            cv2.imwrite(new_images_dir + img_name, img)

            # Second line is the number of faces in the picture
            number_faces = reader.readline()
            writer.write(number_faces)
            
            if number_faces == "0":
                reader.readline()
                continue

            for face in range(int(number_faces)):
                x,y,w,h = [int(elem) for elem in reader.readline().split()[:4]]
                x -= 3
                y -= 3
                end_x = x + w + 6
                end_y = y + h + 6

                # what we'll write is the start and end coords for the region where a face might be
                writer.write(" ".join([str(elem) for elem in [x, y, end_x, end_y]]))
                writer.write("\n")
            counter += int(number_faces)
            if counter > limit_faces:
                return

In [37]:
rewrite_test_file("./Image Resources/Dataset/wider_face_split/wider_face_val_bbx_gt.txt",
                  "./Image Resources/Dataset/WIDER_val/images/", 
                  "./Image Resources/Dataset/WIDER_test/test.txt", 
                  "./Image Resources/Dataset/WIDER_test/images/",
                  20000)

In [ ]:
def compare_results(results_file, expected_file):
    image_faces = {}
    with open(expected_file, "r") as reader:
        while True:
            image_dir = reader.readline()
            if not image_dir: 
                break
            image_faces[image_dir] = []
            num_faces = int(reader.readline())
            for i in range(num_faces):
                image_faces[image_dir].append([int(elem) for elem in reader.readline().rstrip().split()])
    true_pos = 0
    false_pos = 0
    false_neg = 0
    with open(results_file, "r") as reader:
        while True:
            image_dir = reader.readline()
            if not image_dir:
                return true_pos, false_pos, false_neg
            expected_images = image_faces[image_dir]
            num_found_faces = int(reader.readline())
            for i in range(num_found_faces):
                start_x, start_y, width, height = [int(elem) for elem in reader.readline().split()]
                ## In order to be recognized as a face, it has to be within bounds, therfore
                compatible_index = -1
                for i in range(len(expected_images)):
                    expected_face = expected_images[i]
                    if start_x > expected_face[0] and start_y > expected_face[1] \
                    and start_x + width < expected_face[2] and start_y < expected_face[3]:
                        compatible_index = i
                        break
                if compatible_index != -1:
                    true_pos += 1
                    expected_images.pop(compatible_index)
                else:
                    false_pos += 1
            false_neg += len(expected_images)
                
            